In [2]:
import torch
from gensim.models import KeyedVectors
import pyprob #https://github.com/probprog/pyprob #!pip install pyprob
from pyprob import Model 
from pyprob.distributions import Categorical, Uniform, Normal
from pyprob import InferenceEngine
import matplotlib.pyplot as plt

Cannot import dbm.gnu: No module named '_gdbm'


C:\Users\bprys\miniconda3\lib\site-packages\pyprob\util.py:327: UserWarning: Empirical distributions on disk may perform slow because GNU DBM is not available. Please install and configure gdbm library for Python for better speed.
  warnings.warn('Empirical distributions on disk may perform slow because GNU DBM is not available. Please install and configure gdbm library for Python for better speed.')


In [3]:
wv = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

In [4]:
# define global question template variables
color_names = ["red", "green", "blue", "orange", "gray", "yellow"]
non_relational_qs = [
    "What shape is the {} object?",
    "Is the {} object on the left?",
    "Is the {} object on the top?"
]
relational_qs = [
    "What shape is the object closest to the {} object?",
    "What shape is the object furthest from the {} object?",
    "How many objects are the same shape as the {} object?"
]
all_qs = [non_relational_qs, relational_qs]

In [5]:
def text_to_vec(text):
    return torch.tensor([wv[w] for w in text.split() if w in wv]).mean(dim=0)

In [6]:
class QuestionsGen(Model):
    def __init__(self, name="QuestionsModel", opt=None):
        super().__init__(name=name)
        self.opt = opt
        self.colors = 6
        
    def compile_question(self, color, qtype, qsubtype, template):
        # compile a question latent
        color_vec = [0, 0, 0, 0, 0, 0]
        type_vec = [0, 0]
        subtype_vec = [0, 0, 0]
        color_vec[color] = 1
        type_vec[qtype] = 1
        subtype_vec[qsubtype] = 1
        question_vec = color_vec + type_vec + subtype_vec
        
        # get the text of the question
        question_text = all_qs[qtype][qsubtype].format(color_names[color])
        
        return question_text, question_vec
        
    def forward(self):
        c_i = pyprob.sample(Categorical(logits=[1 for _ in range(self.colors)]))
        t_i = pyprob.sample(Categorical(logits=(1,1)))
        st_i = pyprob.sample(Categorical(logits=(1,1,1)))
        tmp_i = pyprob.sample(Categorical(logits=(1,1)))
        cand_q_text, cand_q_latent = self.compile_question(c_i, t_i, st_i, tmp_i)
        pyprob.tag(cand_q_latent, name="question_latent")
        pyprob.tag(cand_q_text, name="question_text")
        cand_vec = text_to_vec(cand_q_text)
        
        pyprob.observe(Normal(cand_vec, 0.0001), name="observed_question")
        return cand_q_text, cand_q_latent

In [7]:
model = QuestionsGen(Model)
posterior_dist = model.posterior(
    num_traces=100,
    inference_engine = InferenceEngine.IMPORTANCE_SAMPLING,
    observe = {'observed_question': text_to_vec(question_Tex)},
    initial_trace=None
)

NameError: name 'question_Tex' is not defined

In [35]:
def recover_question(question_text):
    model = QuestionsGen(Model)
    posterior_dist = model.posterior(
        num_traces=1000,
        inference_engine = InferenceEngine.IMPORTANCE_SAMPLING,
        observe = {'observed_question': text_to_vec(question_text) },
        initial_trace=None
    )
    recovered_question = posterior_dist.sample()["question_text"]
    return recovered_question

In [36]:
def test_question_generalization(true_question, modified_question, colour="blue"):
    true_formatted = true_question.format(colour)
    modified_formatted = modified_question.format(colour)
    recovered_question = recover_question(modified_formatted)
    if recovered_question == true_formatted:
        return True
    else:
        print(f"true question: {true_formatted}")
        print(f"modified question: {modified_formatted}")
        print(f"recovered question: {recovered_question}")
        return False

In [37]:
template_pairs = [
    ("What shape is the {} object?", "What is the shape of the {} thing?"),
    ("Is the {} object on the left?", "Is the {} object on the left-hand side?"),
    ("Is the {} object on the top?", "Is the {} object on the top of the screen?"),
    ("What shape is the object closest to the {} object?", "What shape is the object with the least distance to the {} object?"),
    ("What shape is the object furthest from the {} object?", "What shape is the thing that has the greatest distance to the {} thing?"),
    ("How many objects are the same shape as the {} object?", "How many objects have the same geometric form as the {} object?")
]

In [39]:
n_recovered = 0
n_total = 0

for pair in template_pairs:
    pair_recovered = 0
    pair_total = 0
    for colour in color_names:
        recovered = test_question_generalization(pair[0], pair[1], colour=colour)
        if recovered:
            n_recovered += 1
            pair_recovered += 1
        n_total += 1
        pair_total += 1
    print(f"pair: {pair}")
    print(f"pair accuracy: {pair_recovered}/{pair_total}")
print(f"overall accuracy: {n_recovered}/{n_total}, {n_recovered/n_total}")

Time spent  | Time remain.| Progress             | Trace     | ESS    | Traces/sec
0d:00:00:01 | 0d:00:00:00 | #################### | 1000/1000 |  32.00 | 904.23       
Time spent  | Time remain.| Progress             | Trace     | ESS    | Traces/sec
0d:00:00:01 | 0d:00:00:00 | #################### | 1000/1000 |  23.00 | 899.30       
Time spent  | Time remain.| Progress             | Trace     | ESS    | Traces/sec
0d:00:00:01 | 0d:00:00:00 | #################### | 1000/1000 |  26.00 | 917.05       
Time spent  | Time remain.| Progress             | Trace     | ESS    | Traces/sec
0d:00:00:01 | 0d:00:00:00 | #################### | 1000/1000 |  24.00 | 941.90       
Time spent  | Time remain.| Progress             | Trace     | ESS    | Traces/sec
0d:00:00:01 | 0d:00:00:00 | #################### | 1000/1000 |  27.00 | 941.13       
Time spent  | Time remain.| Progress             | Trace     | ESS    | Traces/sec
0d:00:00:01 | 0d:00:00:00 | #################### | 1000/1000 |  24.00 | 

In [40]:
n_recovered/n_total

0.6944444444444444